<a href="https://colab.research.google.com/github/futugyou/pyproject/blob/master/transformers_demo/rag_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
%pip install langchain-community llama-cpp-python langchain

In [ ]:
# 0. prepare texts
text = """
Interstellar is a 2014 epic science fiction film directed by Christopher Nolan, who co-wrote the screenplay with his brother Jonathan Nolan. It features an ensemble cast led by Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn and Michael Caine. Set in a dystopian future where Earth is suffering from catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

The screenplay had its origins in a script that Jonathan had developed in 2007 and was originally set to be directed by Steven Spielberg. Theoretical physicist Kip Thorne was an executive producer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar. It was Lynda Obst's final film as producer before her death. Cinematographer Hoyte van Hoytema shot it on 35 mm film in the Panavision anamorphic format and IMAX 70 mm. Filming began in late 2013 and took place in Alberta, Klaustur, and Los Angeles. Interstellar uses extensive practical and miniature effects, and the company DNEG created additional visual effects.

Interstellar premiered at the TCL Chinese Theatre on October 26, 2014, and was released in theaters in the United States on November 5, and in the United Kingdom on November 7. In the United States, it was first released on film stock, expanding to venues using digital projectors. The film received generally positive reviews from critics and was a commercial success, grossing
758.6 million worldwide with subsequent releases, making it the tenth-highest-grossing film of 2014. Among its various accolades, Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects.
"""

texts = text.split('.')
texts = [t.strip(" \n") for t in texts]

In [ ]:
# 1. text generation model
from llama_cpp.llama import Llama
from langchain.llms import LlamaCpp

llm_raw = Llama.from_pretrained(
    repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
    filename="*fp16.gguf",
    n_gpu_layers=-1,
    n_ctx=2048,
    verbose=True,
)

llm = LlamaCpp(
    model_path=llm_raw.model_path,
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=True)


In [ ]:
%pip install langchain-huggingface
%pip install faiss-cpu

In [ ]:
#  2. embedding model
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_texts(texts, embedding_model)

In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

# 3. RAG

template = """<|user|>
Relevant information:
{context}

Provider a concise answer the following question using the relevant information provided above:
{question}<|end|>
<|assistant|>
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
)

In [ ]:
rag.invoke("income generated")